In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error


#### CARGAMOS EL ARCHIVO INSTALLS CON FEATURES SACADOS DEL ARCHIVO INSTALLS

In [3]:
installs = pd.read_csv('data/installsConFeatures.csv', low_memory=False, parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,Unnamed: 0,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,top10apps,top50marcas,top10marcas,top10marcasSinNan,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng
created,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,94738,70,1,4432995619177048534,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
2019-04-18 00:00:01.851,94737,70,1,5904733559638204455,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
2019-04-18 00:00:05.152,111877,65,1,896373747754111825,False,True,6287817205707153877,6,102,2,...,3,6,6,6,2,2,1,0,0,8
2019-04-18 00:00:05.589,76146,27,0,3399210824535017892,False,False,6287817205707153877,0,190,2,...,0,0,0,0,21,14,6,1,1,2
2019-04-18 00:00:06.795,435184,339,0,1541425881979513687,False,False,6287817205707153877,0,190,2,...,0,0,0,0,68,46,10,1,1,6


### Dropeamos columnas que quedaron mal

In [4]:
installs.drop(['Unnamed: 0'],axis = 1,inplace=True)

#### Luego cargamos la primer ventana y sus lebels correspondientes a la segunda (Asi con todas las demas)

In [6]:
info1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info1['created'] = info1['created'] - pd.Timestamp('2019-04-21 00:00:00')
info1['created'] = info1['created'].transform(lambda x: x.total_seconds())
info1 = info1.reset_index()
info1 = info1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="right")
info1 = info1.rename(columns={'created_x':'label','created_y':'created'})

label1 = info1['label'].to_frame()
train1 = info1.drop('label', axis=1)

In [8]:
info2 = installs['2019-04-22':'2019-04-24'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info2['created'] = info2['created'] - pd.Timestamp('2019-04-22 00:00:00')
info2['created'] = info2['created'].transform(lambda x: x.total_seconds())
info2 = info2.reset_index()
info2 = info2.merge(installs['2019-04-19':'2019-04-21'].reset_index(),on="ref_hash", how="right")
info2 = info2.rename(columns={'created_x':'label','created_y':'created'})

label2 = info2['label'].to_frame()
train2 = info2.drop('label', axis=1)

In [7]:
info3 = installs['2019-04-23':'2019-04-25'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info3['created'] = info3['created'] - pd.Timestamp('2019-04-23 00:00:00')
info3['created'] = info3['created'].transform(lambda x: x.total_seconds())
info3 = info3.reset_index()
info3 = info3.merge(installs['2019-04-20':'2019-04-22'].reset_index(),on="ref_hash", how="right")
info3 = info3.rename(columns={'created_x':'label','created_y':'created'})

label3 = info3['label'].to_frame()
train3 = info3.drop('label', axis=1)

In [5]:
info4 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info4['created'] = info4['created'] - pd.Timestamp('2019-04-24 00:00:00')
info4['created'] = info4['created'].transform(lambda x: x.total_seconds())
info4 = info4.reset_index()
info4 = info4.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
info4 = info4.rename(columns={'created_x':'label','created_y':'created'})

label4 = info4['label'].to_frame()
train4 = info4.drop('label', axis=1)

#### Como al algoritmo que aplicaremos despues no le gustan los NAN, a esos nan le ponemos 3 dias ( en segundos )

In [9]:
train1 = train1.dropna()
train2 = train2.dropna()
train3 = train3.dropna()
train4 = train4.dropna()
label1 = label1.fillna(3*24*60*60)
label2 = label2.fillna(3*24*60*60)
label3 = label3.fillna(3*24*60*60)
label4 = label4.fillna(3*24*60*60)

#### Ahora sacaremos mas labels para cada ventana en particular...

In [10]:
train1['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-18 00:00:00')).transform(lambda x: x.total_seconds())
train2['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-19 00:00:00')).transform(lambda x: x.total_seconds())
train3['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-20 00:00:00')).transform(lambda x: x.total_seconds())
train4['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-21 00:00:00')).transform(lambda x: x.total_seconds())

In [11]:
train1['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train1))
train2['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train2))
train3['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train3))
train4['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train4))

In [12]:
train1.drop(['created'],axis =1, inplace = True )
train2.drop(['created'],axis =1, inplace = True )
train3.drop(['created'],axis =1, inplace = True )
train4.drop(['created'],axis =1, inplace = True )

In [13]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160861 entries, 0 to 160860
Data columns (total 31 columns):
ref_hash                            160861 non-null int64
application_id                      160861 non-null int64
ref_type                            160861 non-null int64
attributed                          160861 non-null bool
implicit                            160861 non-null bool
device_countrycode                  160861 non-null int64
device_brand                        160861 non-null int64
device_model                        160861 non-null int64
session_user_agent                  160861 non-null int64
user_agent                          160861 non-null int64
kind                                160861 non-null int64
wifi                                160861 non-null bool
ip_address                          160861 non-null int64
device_language                     160861 non-null int64
TopApp/Otro                         160861 non-null int64
dia                   

#### Ahora comenzaremos a entrenar con XGBOOST

In [18]:
train1.shape,label1.shape

((160861, 31), (160861, 1))

In [19]:
data_dmatrix = xgb.DMatrix(data=train1,label=label1)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(train1, label1, test_size=0.2, random_state=123)

In [41]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [42]:
xg_reg.fit(X_train,y_train)


[14:11:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [25]:
preds = xg_reg.predict(X_test)


In [26]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 91630.153102


In [27]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="rmse", as_pandas=True, seed=123)

[13:56:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
cv_results.head()


,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,230783.828125,31.646742,230746.484375,16.341903
1,207941.760417,159.380333,207935.291667,160.948524
2,187375.572917,88.439841,187440.369792,65.174052
3,168978.958333,95.380471,168999.760417,87.610804
4,152367.692708,20.185381,152368.385417,9.316365


In [29]:
print((cv_results["test-rmse-mean"]).tail(1))


49    13515.974284
Name: test-rmse-mean, dtype: float64


In [30]:
xg_reg_ = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)


[13:59:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [10000, 10000]
plt.show()

#### Ahora con el modelo que entrenamos vamos a entrenar las ultima ventana. Y los datos que nos faltan los sacamos de la anteultima ventana.

In [37]:
train4.head()

,ref_hash,application_id,ref_type,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,...,top10marcas,top10marcasSinNan,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng,tiempoDesdeComienzoDeVentana,accionesDesdeQueComenzoLaVentana
0,622102439689666,121,1,False,True,6287817205707153877,5,169,2,198,...,5,5,2,2,1,0,0,8,-259198.440,0
1,622102439689666,121,1,False,False,6287817205707153877,5,169,2,198,...,5,5,2,2,1,1,1,8,-259198.149,1
2,928188758790177,68,0,False,False,6287817205707153877,0,190,2,2376,...,0,0,13,9,5,1,1,3,-259194.848,2
3,4129436938177104,117,1,False,False,6287817205707153877,0,790,0,0,...,0,0,0,0,0,1,1,0,-259194.411,3
4,5133475695351500,194,0,False,False,6287817205707153877,0,190,2,6037,...,0,0,38,26,7,1,1,3,-259193.205,4


In [38]:
X_train.shape, train4.shape

((128688, 31), (160617, 31))

In [43]:
preds = xg_reg.predict(train4)


In [46]:
train4['predicciones']=preds

In [62]:
preds = xg_reg.predict(train3)


In [63]:
train3['predicciones']=preds

#### Ahora que tenemos las predicciones para la ventana 4. Lo que vamos hacer es fijarnos para cuantos refhash del targen coinciden... deberian ser aprox 1500

In [74]:
target = pd.read_csv('data/final.csv', low_memory=False, names=['ref_hash'])


In [75]:
target.head()

,ref_hash
0,1000169251625791246
1,1000395625957344683
2,1003027494996471685
3,1006670001679961544
4,1007573308966476713


In [76]:
mergeados = train4.merge(target, left_on = 'ref_hash',right_on='ref_hash', how='inner')

In [77]:
len(mergeados) #Es un buen numero.

1780

In [78]:
### Ahora vamos a quedarnos con todos los que estan en target. Ya que eso es lo que nos interesa
mergeados = train4.merge(target, left_on = 'ref_hash',right_on='ref_hash', how='right')

In [79]:
len(mergeados)

4431

In [80]:
mergeados.head()

,ref_hash,application_id,ref_type,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,...,top50agentes,top10agentes,kindReducido,top10tipo,top10leng,tiempoDesdeComienzoDeVentana,accionesDesdeQueComenzoLaVentana,predicciones,sc,ref_hash_sc
0,345481582461167855,68.0,0.0,False,False,6.287817e+18,0.0,190.0,2.0,2378.0,...,9.0,5.0,1.0,1.0,3.0,-259002.920,188.0,133642.484375,345481582461167855_sc,345481582461167855_sc
1,345481582461167855,68.0,0.0,False,False,6.287817e+18,0.0,190.0,2.0,2378.0,...,9.0,5.0,1.0,1.0,3.0,-259002.803,189.0,133680.593750,345481582461167855_sc,345481582461167855_sc
2,571264229455029355,97.0,1.0,False,False,6.287817e+18,394.0,2696.0,2.0,3269.0,...,2.0,1.0,1.0,1.0,8.0,-258866.318,309.0,133711.984375,571264229455029355_sc,571264229455029355_sc
3,574356212612010393,194.0,0.0,False,False,6.287817e+18,0.0,190.0,2.0,6037.0,...,26.0,7.0,1.0,1.0,3.0,-258864.477,312.0,133878.453125,574356212612010393_sc,574356212612010393_sc
4,575290294068143648,21.0,1.0,False,False,6.287817e+18,0.0,58.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,-258861.121,314.0,128289.148438,575290294068143648_sc,575290294068143648_sc


In [84]:
mergeados=mergeados.fillna(60*60*24*3)

In [86]:
mergeados=mergeados.groupby('ref_hash').agg({'predicciones':'mean'})

In [90]:
mergeados.head().reset_index().to_csv('data/preds_gxboost_installs_4ventana.csv')

In [92]:
### Ahora para la tercer ventana.
mergeados2 = train3.merge(target, left_on = 'ref_hash',right_on='ref_hash', how='inner')

In [93]:
len(mergeados2)

1764

In [94]:
mergeados2 = train3.merge(target, left_on = 'ref_hash',right_on='ref_hash', how='right')

In [95]:
mergeados2=mergeados2.fillna(60*60*24*3)

In [96]:
mergeados2=mergeados2.groupby('ref_hash').agg({'predicciones':'mean'})

In [97]:
mergeados2.head().reset_index().to_csv('data/preds_gxboost_installs_3ventana.csv')